# Tâches NLP

Ce notebook présente les cas d'usages les plus fréquents en utilisant la librairie Transformers : question answering, sequence classification, named entity recognition etc.

Ces exemples utilisent les pipelines et les classes Auto*. Ces classes vont créer une instance d'un modèle en fonction d'un checkpoint, en sélectionnant automatiquement l'architecture du modèle.
Pour en savoir plus, rendez-vous sur la documentation d'[AutoModel](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModel).

Les checkpoints sont généralement pré-entrainés sur de grand jeu de données et fine-tunés pour une tâche spécifique. Les limites sont :

- Tous les modèles ne sont pas fine-tunés sur toutes les tâches.
- Les modèles ont été fine-tunés sur un jeu de données spécifique. Ce jeu de données peut⁻être différent de votre cas d'usage.

Note : si un checkpoint demandé n'est pas disponible pour une tâche donnée, seul le transformer de base sera chargé et la partie du modèle spécifique à la tâche sera initialisée avec des poids aléatoires (produisant des sorties aléatoires).

## Sequence Classification

La classification de séquence consiste à classifier des séquences selon plusieurs classes (voir le dataset [GLUE](https://huggingface.co/datasets/glue)).

Ici nous utilisons un checkpoint pour déterminer si deux séquences sont une paraphrase l'une de l'autre. Le modèle requiert alors que nous lui fournissions une liste de dictionnaire de pair de texte. La sortie est `LABEL_0` et `LABEL_1` pour indiquer respectivement que ce n'est pas une paraphrase et que c'est une paraphrase :

In [ ]:
from transformers import pipeline

paraphrase_or_not = pipeline("text-classification", model="bert-base-cased-finetuned-mrpc")

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

text_pairs = [
    {"text": sequence_0, "text_pair": sequence_1},
    {"text": sequence_0, "text_pair": sequence_2}
]
results = paraphrase_or_not(text_pairs)
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

Ci-dessous le même exemple utilisant les Auto* classes. Les étapes sont les suivantes :

1. Créer une instance de tokenizer et d'un modèle depuis un checkpoint.
2. Créer une séquence depuis deux phrases, en utilisant les séparateurs, le type de tokens et les masques d'attention attendus par le modèle
3. Envoyer la séquence au modèle
4. Calculer le softmax du résultat pour obtenir des probabilités
5. Afficher le résultat

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ["not paraphrase", "is paraphrase"]

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

# The tokenizer will automatically add any model specific separators (i.e. <CLS> and <SEP>) and tokens to
# the sequence, as well as compute the attention masks.
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="pt")

paraphrase_classification_logits = model(**paraphrase).logits
not_paraphrase_classification_logits = model(**not_paraphrase).logits

paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]

# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")

In [ ]:
# Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

## Extractive Question Answering

L'Extractive Question Answering consiste à extraire une réponse depuis un texte pour une question donnée. Un dataset usuel pour cette tâche est le [SQuAD dataset](https://huggingface.co/datasets/squad).

Voici un exemple de pipeline qui permet de réaliser une tâche de question answering à l'aide d'un modèle fine-tuné sur SQuAD :

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")

context = r"""
BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages.
"""

This returns an answer extracted from the text, a confidence score, alongside "start" and "end" values, which are the
positions of the extracted answer in the text.

In [ ]:
result = question_answerer(question="How many programming languages does BLOOM support?", context=context)
print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"
)

In [ ]:
result = question_answerer(question="What can BLOOM do?", context=context)
print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"
)

Nous pouvons faire de même avec un modèle et un tokenizer :

1. Créer une instance d'un tokenizer et d'un modèle depuis un checkpoint
2. Définir un contexte et des questions
3. Itérer sur les questions et construire une séquence avec le texte et la question courante, avec le séparateur du modèle, les bons types de tokens et le masque d'attention
4. Inférer la réponse avec le modèle, la sortie contient des scores pour chaque token du contexte pour indiquer le début et la fin de la réponse à extraire
5. Calculer l'argmax des résultats
6. Récupérer le token de début et de fin de la réponse et les convertir en texte
7. Afficher le résultat

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning of answer with the argmax of the score
    answer_start = torch.argmax(answer_start_scores)
    # Get the most likely end of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )

    print(f"Question: {question}")
    print(f"Answer: {answer}")

## Language Modeling

Le Language modeling consiste à entrainer un modèle sur un corpus documentaire (pouvant être lié à un domaine précis). Généralement, les modèles de transformers sont entrainés en utilisant le language modeling (BERT avec du  masked language modeling, GPT-2 avec du causal language modeling).

Le Language modeling peut être utile en dehors du pré-entrainement, notamment pour adapter un modèle à un autre domaine. Par exemple en prenant un modèle entrainé sur un vaste corpus documentaire et en l'adaptant sur un dataset d'articles scientifiques (voir [LysandreJik/arxiv-nlp](https://huggingface.co/lysandre/arxiv-nlp)).

### Masked Language Modeling

Le Masked Language Modeling consiste à cacher un ou plusieurs tolens dans une séquence, puis à utiliser un modèle pour inférer les tokens les plus appropriés pour les tokens masqués. Cela permet au modèle de porter son attention aussi en amont et en aval du ou des tokens cachés. Ce type de modèle et l'entrainement associé permet de créer un base robuste pour des tâches en aval tel que l'Extractive Question Answering (voir [Lewis, Lui, Goyal et al.](https://arxiv.org/abs/1910.13461), part 4.2).

Ci-dessous, un exemple de pipeline pour remplacer un masque dans une séquence :

In [ ]:
from transformers import pipeline

unmasker = pipeline("fill-mask")

This outputs the sequences with the mask filled, the confidence score, and the token id in the tokenizer vocabulary:

In [ ]:
from pprint import pprint

pprint(
    unmasker(
        f"HuggingFace is creating a {unmasker.tokenizer.mask_token} that the community uses to solve NLP tasks."
    )
)

Ci-dessous, du masked language modeling en utilisant un modèle et un tokenizer :

1. Créer une instance du modèle et du tokenizer depuis un nom de checkpoint
2. Définir une séquence avec un token caché (en utilisant `tokenizer.mask_token`)
3. Encoder la séquence en une liste d'IDs et trouvé la position du token masqué dans la liste
4. Récupérer les prédictions pour l'index du token masqué. Ces prédictions contiennent un score pour chaque token du vocabulaire du modèle. Les scores les plus élevés correspondent aux tokens les plus probables
5. Récupérer les 5 tokens les plus probables avec la fonction PyTorch `topk`
6. Remplacer le masque par les tokens et afficher le résultat

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-cased")

sequence = (
    "Distilled models are smaller than the models they mimic. Using them instead of the large "
    f"versions would help {tokenizer.mask_token} our carbon footprint."
)

inputs = tokenizer(sequence, return_tensors="pt")
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

token_logits = model(**inputs).logits
mask_token_logits = token_logits[0, mask_token_index, :]
print("mask_token_logits shape", mask_token_logits.shape)

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
torch.topk

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

### Causal Language Modeling

Le Causal language modeling consiste à prédire le token le plus probable complétant une séquence donnée. Ici le modèle ne portera son attention qu'aux tokens situés en amont du masque. Ce type de modèle est utile pour les tâches génératives.

Ci-dessous, un exemple d'utilisation d'un modèle et d'un tokenizer en utilisant la méthode [top_k_top_p_filtering()](https://huggingface.co/docs/transformers/main/en/internal/generation_utils#transformers.top_k_top_p_filtering) pour récupérer un le token le plus probable.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch import nn

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

sequence = f"Hugging Face is based in DUMBO, New York City, and"

inputs = tokenizer(sequence, return_tensors="pt")
input_ids = inputs["input_ids"]

# get logits of last hidden state
next_token_logits = model(**inputs).logits[:, -1, :]

# filter
filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)

# sample
probs = nn.functional.softmax(filtered_next_token_logits, dim=-1)
next_token = torch.multinomial(probs, num_samples=1)

generated = torch.cat([input_ids, next_token], dim=-1)

resulting_string = tokenizer.decode(generated.tolist()[0])
print(resulting_string)

In [ ]:
next_token_logits.shape

This outputs a (hopefully) coherent next token following the original sequence, which in our case is the word *is* or
*features*.

In the next section, we show how [generation.GenerationMixin.generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) can be used to
generate multiple tokens up to a specified length instead of one token at a time.

### Text Generation

La generation de texte consiste à créer une séquence de texte cohérente à partir d'un contexte (aka prompt). L'exemple suivant montre l'utilisation d'un pipeline pour la génération de texte en utilisant GPT-2 :

In [ ]:
from transformers import pipeline

text_generator = pipeline("text-generation")
print(text_generator("As far as I am concerned, I will", max_length=50, do_sample=False))

Nous avons généré un texte de 50 tokens maximum. Le pipeline utilise la méthode [PreTrainedModel.generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) pour générer du texte. Les arguments par défaut de cette méthode peuvent être écrasés par les arguments du pipeline (`max_length` et `do_sample` par exemple).

Ci-dessous, nous générons du texte avec le modèle `XLNet` en utilisant la méthode `generate()` :

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("xlnet-base-cased")
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

# Padding text helps XLNet with short prompts - proposed by Aman Rusia in https://github.com/rusiaaman/XLNet-gen#methodology
PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""

prompt = "Today the weather is really nice and I am planning on "
inputs = tokenizer(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]

prompt_length = len(tokenizer.decode(inputs[0]))
outputs = model.generate(inputs, max_length=250, do_sample=True, top_p=0.95, top_k=60)
generated = prompt + tokenizer.decode(outputs[0])[prompt_length + 1 :]

print(generated)

La génération de texte est possible avec *GPT-2*, *OpenAi-GPT*, *CTRL*, *XLNet*, *Transfo-XL* et *Reformer*. Les modèles *XLNet* et *Transfo-XL* nécessitent généralement du padding pour fonctionner correctement.

## Named Entity Recognition

La reconnaissance d'entités nommées (Named Entity Recognition, NER) consiste à classifier les tokens d'une séquence selon plusieurs classes. Par exemple, pour identifier une personne, une entreprise ou un lieu.

Ci-dessous nous utilisons un pipeline de NER pour identifier les entités suivantes :

- O, Outside of a named entity
- B-MIS, Beginning of a miscellaneous entity right after another miscellaneous entity
- I-MIS, Miscellaneous entity
- B-PER, Beginning of a person's name right after another person's name
- I-PER, Person's name
- B-ORG, Beginning of an organisation right after another organisation
- I-ORG, Organisation
- B-LOC, Beginning of a location right after another location
- I-LOC, Location

Ce modèle a été entrainé sur le dataset CoNLL-2003.

In [ ]:
from transformers import pipeline

ner_pipe = pipeline("ner")

sequence = """Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO,
therefore very close to the Manhattan Bridge which is visible from the window."""

This outputs a list of all words that have been identified as one of the entities from the 9 classes defined above.
Here are the expected results:

In [ ]:
for entity in ner_pipe(sequence):
    print(entity)


Note how the tokens of the sequence "Hugging Face" have been identified as an organisation, and "New York City",
"DUMBO" and "Manhattan Bridge" have been identified as locations.

Nous pouvons aussi utiliser les Auto* classes pour réaliser cette NER :

1. Créer une instance d'un modèle et d'un tokenizer depuis un checkpoint
2. Définir un texte avec des entités nommées connues
3. Encoder la séquence en tokens
4. Récupérer les prédictions, la sortie sera une distribution pour chacune des 9 classes du modèle. Transformer la sortie en récupérant la classe la plus probable pour chaque token avec argmax
6. Afficher la classe prédite pour chaque token

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = (
    "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, "
    "therefore very close to the Manhattan Bridge."
)

inputs = tokenizer(sequence, return_tensors="pt")
tokens = inputs.tokens()

outputs = model(**inputs).logits
predictions = torch.argmax(outputs, dim=2)

Contrairement au pipeline, la sortie contient une prédiction pour chaque token (classe "O" inclus).

Les prédictions sont un entier correspondant à la classe prédite. Nous pouvons utiliser `model.config.id2label` pour récupérer le nom de la classe :

In [ ]:
for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

## Summarization

La Summarization consiste à résumer un document ou un article en un texte plus court.

Ci-dessous un exemple de pipeline qui résume un texte en utilisant le modèle Bart (entrainé sur le dataset CNN / Daily Mail data set qui contient des articles de ces deux sources et des résumés) :

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

Le pipeline de summarization utilisant la méthode `PreTrainedModel.generate()`, nous pouvons définir certains paramètres tel que `max_length` et `min_length` :

In [ ]:
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

Nous pouvons réaliser de la summarization en utilisant un modèle et son tokenizer, le processus est le suivant :

1. Créer une instance d'un modèle (tel que `Bart` ou `T5`) et d'un tokenizer pour un chackpoint
2. Définir un text a résumer
3. Ajouter le préfixe "summarize: " au texte
4. Utiliser la méthode `PreTrainedModel.generate()`pour générer un résumé

Dans cet exemple, nous utilisons le modèle T5 de Google, qui bien qu'entrainé sur un vaste corpus (incluant CNN / Daily Mail) fournit de très bons résultats.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer("summarize: " + ARTICLE, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## Translation

Nous pouvons aussi utiliser un modèle génératif tel que T5 pour traduire des textes. Ce pipeline utilisant la méthode `PreTrainedModel.generate()` nous pouvons définir une valeur pour le paramètre `max_length` :

In [ ]:
from transformers import pipeline

translator = pipeline("translation_en_to_fr")
print(translator("Hugging Face is a technology company based in New York and Paris", max_length=40))

Pour traduire un texte en utilisant un modèle et son tokenizer avec les Auto* classes, il faut :

1. Créer une instance d'un modèle et d'un tokenizer à partir d'un checkpoint
2. Définir un text a traduire
3. Ajouter le préfixe "translate English to French: " au texte
4. Utiliser la méthode PreTrainedModel.generate()pour générer un résumé

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

inputs = tokenizer(
    "translate English to French: Hugging Face is a technology company based in New York and Paris",
    return_tensors="pt",
)
outputs = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## Zero shot classification

Certains modèles peuvent réaliser de la classification sans entrainement (zero shot learning) :

In [ ]:
pipe = pipeline(model="facebook/bart-large-mnli")
pipe(
    "I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
)